# Carsen Essing Machine Learning Assignment 2

## Upload data- LA 3

In [15]:
import pandas as pd

# Read your dataset
#df = pd.read_csv("LAQ3.csv")

# Print all columns to inspect the dataset
#print("All columns in the dataset:")
#print(df.columns.tolist())

# Define the list of columns to keep, using 'neighbourhood_cleansed' instead of 'neighbourhood'
#columns_to_keep = [
    'id',                   # unique identifier
    'property_type',        # property type (e.g., Apartment, House)
    'room_type',            # room type (e.g., Entire home/apt, Private room)
    'accommodates',         # number of guests the listing can accommodate
    'bedrooms',             # number of bedrooms
    'bathrooms',            # number of bathrooms
    'beds',                 # number of beds
    'price',                # price per night (target variable)
    'minimum_nights',       # minimum nights required
    'maximum_nights',       # maximum nights allowed
    'amenities',            # amenities (raw string; later processed)
    'number_of_reviews',    # count of reviews
    'reviews_per_month',    # review frequency
    'review_scores_rating', # overall review score
    'latitude',             # geographic coordinate
    'longitude',            # geographic coordinate
    'neighbourhood_cleansed' # cleaned neighbourhood name
]

# Identify the columns in the dataset that are not in the keep list
#columns_to_drop = [col for col in df.columns if col not in columns_to_keep]

#print("\nColumns recommended to drop:")
#print(columns_to_drop)

# Create a reduced DataFrame with only the desired columns
#df_reduced = df[columns_to_keep]

# Save the reduced DataFrame to a new CSV file for easier sharing
#df_reduced.to_csv("LAQ3_reduced.csv", index=False)
#print("\nThe reduced CSV file 'LAQ3_reduced.csv' has been created.")


All columns in the dataset:
['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', '

In [17]:
import zipfile

# Specify the CSV file to compress and the desired zip file name
csv_filename = "LAQ3_reduced.csv"
zip_filename = "LAQ3_reduced.zip"

# Create a ZipFile object in write mode, compressing with ZIP_DEFLATED
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add the CSV file to the zip archive. The arcname parameter ensures that only the filename is stored (not the entire path)
    zipf.write(csv_filename, arcname=csv_filename)

print(f"{csv_filename} has been successfully compressed into {zip_filename}")


LAQ3_reduced.csv has been successfully compressed into LAQ3_reduced.zip


In [22]:
#SC 4 Data upload

#import pandas as pd

# Read your dataset
#df = pd.read_csv("LAQ4.csv")

# Print all columns to inspect the dataset
#print("All columns in the dataset:")
#print(df.columns.tolist())

# Define the list of columns to keep, using 'neighbourhood_cleansed' instead of 'neighbourhood'
#columns_to_keep = [
    'id',                   # unique identifier
    'property_type',        # property type (e.g., Apartment, House)
    'room_type',            # room type (e.g., Entire home/apt, Private room)
    'accommodates',         # number of guests the listing can accommodate
    'bedrooms',             # number of bedrooms
    'bathrooms',            # number of bathrooms
    'beds',                 # number of beds
    'price',                # price per night (target variable)
    'minimum_nights',       # minimum nights required
    'maximum_nights',       # maximum nights allowed
    'amenities',            # amenities (raw string; later processed)
    'number_of_reviews',    # count of reviews
    'reviews_per_month',    # review frequency
    'review_scores_rating', # overall review score
    'latitude',             # geographic coordinate
    'longitude',            # geographic coordinate
    'neighbourhood_cleansed' # cleaned neighbourhood name
]

# Identify the columns in the dataset that are not in the keep list
columns_to_drop = [col for col in df.columns if col not in columns_to_keep]

print("\nColumns recommended to drop:")
print(columns_to_drop)

# Create a reduced DataFrame with only the desired columns
df_reduced = df[columns_to_keep]

# Save the reduced DataFrame to a new CSV file for easier sharing
df_reduced.to_csv("LAQ4_reduced.csv", index=False)
print("\nThe reduced CSV file 'LAQ4_reduced.csv' has been created.")

All columns in the dataset:
['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', '

In [24]:
import zipfile

# Specify the CSV file to compress and the desired zip file name
csv_filename = "LAQ4_reduced.csv"
zip_filename = "LAQ4_reduced.zip"

# Create a ZipFile object in write mode, compressing with ZIP_DEFLATED
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add the CSV file to the zip archive. The arcname parameter ensures that only the filename is stored (not the entire path)
    zipf.write(csv_filename, arcname=csv_filename)

print(f"{csv_filename} has been successfully compressed into {zip_filename}")


LAQ4_reduced.csv has been successfully compressed into LAQ4_reduced.zip


# Code starts here ! Upload Q3 LA Data from GitHub

In [33]:
import requests
import zipfile
import io
import pandas as pd

# Raw URL to your zip file on GitHub
url = "https://raw.githubusercontent.com/CarsenEssing/Essing_ML_Assignment2/main/LAQ3_reduced.zip"

# Download the file as bytes
response = requests.get(url)
response.raise_for_status()  # Raises an error if the download fails

# Open the downloaded content as a ZipFile
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    # List the files in the zip archive (for verification)
    print("Files in the zip:", z.namelist())
    # Assuming the CSV file inside is named 'LAQ3_reduced.csv'
    df = pd.read_csv(z.open("LAQ3_reduced.csv"))

print("Data loaded successfully:")
print(df.head())


Files in the zip: ['LAQ3_reduced.csv']
Dataset shape: (45533, 17)
Columns: ['id', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'bathrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights', 'amenities', 'number_of_reviews', 'reviews_per_month', 'review_scores_rating', 'latitude', 'longitude', 'neighbourhood_cleansed']
                    id         property_type        room_type  accommodates  \
0   670339032744709144          Entire condo  Entire home/apt             6   
1             37014494    Entire rental unit  Entire home/apt             2   
2  1024835174766068422           Entire home  Entire home/apt             6   
3   850744632375448560  Private room in home     Private room             2   
4   953950676345326970    Entire rental unit  Entire home/apt             2   

   bedrooms  bathrooms  beds    price  minimum_nights  maximum_nights  \
0       3.0        3.0   3.0  $399.00              30             365   
1       2.0        NaN   NaN      NaN      

# Data Wrangling

## Clean up price and amenities columns

In [37]:
# Remove the dollar sign and any commas, then convert the string to a float.
df['price_clean'] = df['price'].replace({r'\$': '', ',': ''}, regex=True).astype(float)

# Print the original and cleaned price columns to verify changes.
print("Price before and after cleaning:")
print(df[['price', 'price_clean']].head())

# Process the 'amenities' column:
# Remove the enclosing square brackets and quotes, then split the string into a list.
df['amenities_list'] = df['amenities'].apply(lambda x: x.strip('[]"').split('", "'))

# Print a sample of the processed amenities column.
print("\nSample of processed amenities column:")
print(df[['amenities', 'amenities_list']].head())


Price before and after cleaning:
     price  price_clean
0  $399.00        399.0
1      NaN          NaN
2  $434.00        434.0
3   $49.00         49.0
4  $231.00        231.0

Sample of processed amenities column:
                                           amenities  \
0  ["Dedicated workspace", "Air conditioning", "K...   
1  ["Air conditioning", "Pets allowed", "Hot wate...   
2  ["Air conditioning", "Kitchen", "Carbon monoxi...   
3  ["Dedicated workspace", "Air conditioning", "K...   
4  ["Clothing storage: closet", "Shared beach acc...   

                                      amenities_list  
0  [Dedicated workspace, Air conditioning, Kitche...  
1  [Air conditioning, Pets allowed, Hot water, Ki...  
2  [Air conditioning, Kitchen, Carbon monoxide al...  
3  [Dedicated workspace, Air conditioning, Kitche...  
4  [Clothing storage: closet, Shared beach access...  


## create bianary code for popular amenities

In [39]:
# Define a list of popular amenities to extract as binary features.
# You may adjust this list based on what appears in your dataset.
popular_amenities = [
    "Air conditioning", 
    "Wifi", 
    "Kitchen", 
    "Heating", 
    "Washer", 
    "Dryer", 
    "TV", 
    "Cable TV", 
    "Free parking on premises", 
    "Pool", 
    "Hot water"
]

# For each amenity, create a new binary column.
for amenity in popular_amenities:
    # Create a column name by replacing spaces with underscores and converting to lowercase.
    col_name = f"amenity_{amenity.replace(' ', '_').lower()}"
    df[col_name] = df["amenities_list"].apply(lambda x: 1 if amenity in x else 0)

# Display the first few rows of the new binary amenity columns.
binary_columns = [f"amenity_{amenity.replace(' ', '_').lower()}" for amenity in popular_amenities]
print("Binary amenity columns:")
print(df[binary_columns].head())


Binary amenity columns:
   amenity_air_conditioning  amenity_wifi  amenity_kitchen  amenity_heating  \
0                         1             1                1                0   
1                         1             1                1                1   
2                         1             1                1                0   
3                         1             1                1                0   
4                         1             1                1                1   

   amenity_washer  amenity_dryer  amenity_tv  amenity_cable_tv  \
0               1              0           1                 0   
1               1              1           1                 0   
2               1              0           1                 0   
3               1              0           0                 0   
4               0              0           0                 0   

   amenity_free_parking_on_premises  amenity_pool  amenity_hot_water  
0                                

In [59]:
# Calculate missing percentages for each column
missing_percentages = (df.isnull().sum() / df.shape[0]) * 100
print("Missing percentages per column:")
print(missing_percentages.sort_values(ascending=False))


Missing percentages per column:
id                                  0.0
property_type                       0.0
amenity_pool                        0.0
amenity_free_parking_on_premises    0.0
amenity_cable_tv                    0.0
amenity_tv                          0.0
amenity_dryer                       0.0
amenity_washer                      0.0
amenity_heating                     0.0
amenity_kitchen                     0.0
amenity_wifi                        0.0
amenity_air_conditioning            0.0
amenities_list                      0.0
price_clean                         0.0
neighbourhood_cleansed              0.0
longitude                           0.0
latitude                            0.0
review_scores_rating                0.0
reviews_per_month                   0.0
number_of_reviews                   0.0
amenities                           0.0
maximum_nights                      0.0
minimum_nights                      0.0
price                               0.0
beds    

## Ger rid of missing values

In [61]:

# Drop rows where the 'price_clean' value is missing
df = df.dropna(subset=['price_clean'])

# Filter the DataFrame to keep only rows with room_type "Entire home/apt"
df = df[df['room_type'] == "Entire home/apt"]

# Drop rows where 'bedrooms', 'bathrooms', or 'beds' are missing
df = df.dropna(subset=['bedrooms', 'bathrooms', 'beds'])

# Impute missing values for 'reviews_per_month' and 'review_scores_rating' with 0
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)
df['review_scores_rating'] = df['review_scores_rating'].fillna(0)

print("Missing values per column after filtering:")
print(df.isnull().sum())



Missing values per column after filtering:
id                                  0
property_type                       0
room_type                           0
accommodates                        0
bedrooms                            0
bathrooms                           0
beds                                0
price                               0
minimum_nights                      0
maximum_nights                      0
amenities                           0
number_of_reviews                   0
reviews_per_month                   0
review_scores_rating                0
latitude                            0
longitude                           0
neighbourhood_cleansed              0
price_clean                         0
amenities_list                      0
amenity_air_conditioning            0
amenity_wifi                        0
amenity_kitchen                     0
amenity_heating                     0
amenity_washer                      0
amenity_dryer                       0
amenity

# Start predictive model prep: Traianing and Testing Sets